In [3]:
import numpy as np
import gc
import copy
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier


from feat_loading import load_dataframe, load_extra_feats_post

c:\users\yyq\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
app_train = load_dataframe("train_all.csv")
app_test = load_dataframe("test_all.csv")
app_train, app_test = load_extra_feats_post(app_train, app_test)



D:\homecredit\feat_loading.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  'FLAG_DOCUMENT_21']
D:\homecredit\feat_loading.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  new['credit_minus_goods'] = df['AMT_CREDIT'] - df['AMT_GOODS_PRICE']
D:\homecredit\feat_loading.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

In [10]:
for dsname, ds in {"app_train": app_train, "app_test":app_test}.items():
    for col in ds:
#         print("dealing ", dsname, col)
        if ds[col].dtype == 'float32':
            if ds[np.isnan(ds[col])].any():
                print("nan: ", dsname, col)
            elif ds[np.isinf(ds[col])].any():
                print("inf: ", dsname, col)

In [4]:


# rf_settings = dict(
#     n_estimators=10000,
#     max_depth=10,
#     min_samples_leaf=20,
#     min_samples_split=10,
#     min_impurity_decrease=1e-6,
#     n_jobs=4,
#     random_state=50
# )
rf = RandomForestClassifier(
#     **rf_settings
)
rf.fit(app_train[:4000], app_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [9]:
w=pd.DataFrame({"e":[1], "r":[0]})
np.isinf(w["e"]/w["r"]).any()

True

In [12]:
from sklearn.utils.validation import check_array
